# Azure ML Studio Tutorial: Image Classification with CIFAR-10

This tutorial walks you through training a simple convolutional neural network (CNN) on the CIFAR-10 dataset using PyTorch in Azure ML Studio.

We will:
- Load and preprocess the CIFAR-10 dataset
- Define a basic CNN model
- Train the model on Azure ML
- Log metrics and register the model

This notebook is designed to be run inside an **Azure ML Compute Instance** or with a **Compute Cluster** using an Azure ML experiment.


In [1]:
!pip install --upgrade azure-ai-ml torchvision torch matplotlib numpy

import os
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from azureml.core import Workspace, Experiment, Run
from azureml.core.model import Model

print("Libraries imported.")


ModuleNotFoundError: No module named 'azureml'

## Load and Preprocess CIFAR-10 Dataset

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

classes = trainset.classes
print("Classes:", classes)


## Define CNN Model

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)


## Define Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


## Train the Model and Log with Azure ML

In [ ]:
run = Run.get_context()

for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(trainloader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")
    run.log("loss", avg_loss)

print('Finished Training')


## Save and Register the Model

In [ ]:
os.makedirs("outputs", exist_ok=True)
torch.save(model.state_dict(), "outputs/cifar_model.pth")

run.upload_file("outputs/cifar_model.pth", "outputs/cifar_model.pth")
run.complete()


*Check Predictions *

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

# Helper: plot a few test images with predictions
def show_predictions(model, testloader, classes, device, num_images=5):
    model.eval()
    images_shown = 0

    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            for i in range(len(images)):
                plt.imshow(np.transpose(images[i].cpu().numpy(), (1, 2, 0)))
                plt.title(f"Predicted: {classes[preds[i]]}, Actual: {classes[labels[i]]}")
                plt.axis('off')
                plt.show()

                images_shown += 1
                if images_shown >= num_images:
                    return

# Call this function to visualize predictions
# show_predictions(model, test_loader, classes, device)


Plot Metrics

In [ ]:

# Plot loss
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss over Epochs')
plt.legend()
plt.show()

# Plot accuracy
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()
plt.show()


In [ ]:

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get all predictions and labels
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=classes)
disp.plot(xticks_rotation='vertical')
plt.title("Confusion Matrix")
plt.show()


In [ ]:

correct = [0] * len(classes)
total = [0] * len(classes)

model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        for label, prediction in zip(labels, predicted):
            total[label] += 1
            if label == prediction:
                correct[label] += 1

class_accuracy = [100 * c / t if t != 0 else 0 for c, t in zip(correct, total)]

plt.figure(figsize=(10,5))
plt.bar(classes, class_accuracy)
plt.ylabel('Accuracy (%)')
plt.title('Per-class Accuracy')
plt.xticks(rotation=45)
plt.show()
